# Creation of control variables tables

The objective is to develop comprehensive tables containing aggregated control variables for towns (COM) and Zone d'Emploi spanning from 2014 to 2023.

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import pyarrow.feather as feather
import requests
import os
from zipfile import ZipFile

### Import translation table

In [2]:
path = "C:/Users/lujul/OneDrive/Documents/Github/interestnpy"
IDENT_DF = pd.read_feather(path+"/data/interim/tble_de_passage_py.feather").drop_duplicates()

In [3]:
IDENT_DF.head()

,COM,ZE,LIB_COM,LIB_ZE,EPCI,LIB_EPCI,CODGEO_2023,CODGEO_2022,CODGEO_2021,CODGEO_2020,CODGEO_2019,CODGEO_2018,CODGEO_2017,CODGEO_2016,CODGEO_2015,CODGEO_2014
0,13201,9312.0,Marseille 01,Marseille,13201,Marseille 01,13201,13201,13201,13201,13201,13201,13201,13201,13201,13201
1,13202,9312.0,Marseille 02,Marseille,13202,Marseille 02,13202,13202,13202,13202,13202,13202,13202,13202,13202,13202
2,13203,9312.0,Marseille 03,Marseille,13203,Marseille 03,13203,13203,13203,13203,13203,13203,13203,13203,13203,13203
3,13204,9312.0,Marseille 04,Marseille,13204,Marseille 04,13204,13204,13204,13204,13204,13204,13204,13204,13204,13204
4,13205,9312.0,Marseille 05,Marseille,13205,Marseille 05,13205,13205,13205,13205,13205,13205,13205,13205,13205,13205


In [4]:
# Change type of column 'ZE' from float to string
IDENT_DF['ZE'] = IDENT_DF['ZE'].astype(int).astype(str).str.zfill(4)

## 1- Revenues

### Download income data from the Filosofi database for the years 2014 to 2020

The Filosofi database contains data on income and poverty in France for different geographical levels (région, département, EPCI, commune, zone d'emploi...) on a specific year. Depending on the year, the geographical division used by the Filosofi database varies. For example, the 2020 data are proposed in the geography in effect as of 2023, while the 2014 data are only available in the geography in effect as of 2015.

In [5]:
d_urls = {
    "2020" : "https://www.insee.fr/fr/statistiques/fichier/6692392/base-cc-filosofi-2020_XLSX.zip",
    "2019" : "https://www.insee.fr/fr/statistiques/fichier/6036902/base-cc-filosofi-2019_XLSX.zip",
    "2018" : "https://www.insee.fr/fr/statistiques/fichier/5009236/base-cc-filosofi-2018_XLSX-geo2021.zip",
    "2017" : "https://www.insee.fr/fr/statistiques/fichier/4507225/base-filosofi-2017_XLSX.zip",
    "2016" : "https://www.insee.fr/fr/statistiques/fichier/4190004/filosofi-revenu-pauvrete-menage-2016.zip",
    "2015" : "https://www.insee.fr/fr/statistiques/fichier/3560121/filo-revenu-pauvrete-menage-2015.zip",
    "2014" : "https://www.insee.fr/fr/statistiques/fichier/3126432/filo-revenu-pauvrete-menage-2014.zip"
}

In [6]:
def import_filosofi_one_year(year, geo_level, dict_url = d_urls):
    """
    geo_level : "COM" or "ZE"
    """  
    # Download and extract zip file
    url = dict_url[year]
    zipname = url.split("/")[-1]
    r = requests.get(url)
    data_path = path+"/data/external/Revenues/"
    os.makedirs(data_path, exist_ok=True)
    open(data_path + zipname, "wb").write(r.content)    
    with ZipFile(data_path + zipname, 'r') as myzip:
        file_path = [file for file in myzip.namelist() if "filosofi" in file][0]
        data = myzip.open(file_path)
        
        if geo_level == "COM":
            df = pd.read_excel(data, sheet_name="COM", skiprows=5, dtype=str)
        elif geo_level == "ZE":
            sheet_name_ze = "ZE2020" if int(year) >= 2018 else "ZE2010"
            df = pd.read_excel(data, sheet_name=sheet_name_ze, skiprows=5, dtype=str)
    
    return df

### Filosofi data processing

In [7]:
def filosofi_processing_one_year(year, geo_level, dict_url=d_urls): 
    # Import filosofi data
    df = import_filosofi_one_year(year, geo_level, dict_url=d_urls)
        
    # Keep only the columns CODGEO, year and MED{year} (median income)
    df = df.rename(columns={df.columns[4]: f'MED{year}'})   
    df = df[['CODGEO', f'MED{year}']]

    # Change columns type
    df['CODGEO'] = df['CODGEO'].astype(str)
    df[f'MED{year}'] = pd.to_numeric(df[f'MED{year}'], errors='coerce')

    # Merge with the translation table IDENT_DF 
    if geo_level == "COM":
        year_geo = str(int(year)+1) if year in ["2014", "2015"] else str(int(year)+3)
        df = df.merge(IDENT_DF, left_on='CODGEO', right_on=f'CODGEO_{year_geo}', how='right')   
    elif geo_level == "ZE":
        df = df.merge(IDENT_DF, left_on='CODGEO', right_on='ZE', how='right')
    df = df[[geo_level, f'MED{year}']].drop_duplicates()

    # Take the mean of MED{year} for rows with the same 'COM' or 'ZE'
    df = df.groupby(geo_level)[f'MED{year}'].mean().reset_index()
    
    return df

### Merge median disposable income from 2014 to 2020

In [8]:
def filosofi_all_years(geo_level, dict_url=d_urls):
    dfs = [filosofi_processing_one_year(year, geo_level, dict_url) for year in dict_url.keys()]
    df = dfs[0]
    for i in range(1,len(dfs)):
        df = df.merge(dfs[i], on=geo_level)
    return df

#### a) COM level

In [9]:
df_rev_com = filosofi_all_years("COM").dropna().reset_index(drop=True)

In [10]:
df_rev_com

,COM,MED2020,MED2019,MED2018,MED2017,MED2016,MED2015,MED2014
0,01001,24820.0,23970.0,25350.0,23310.0,22679.000000,22228.000000,21576.666667
1,01002,23220.0,23640.0,23890.0,24290.0,24382.083333,22883.333333,21672.857143
2,01004,21070.0,20510.0,20420.0,19860.0,19721.000000,19735.200000,19756.111111
3,01005,24380.0,24050.0,24270.0,23370.0,23378.000000,23182.666667,23204.800000
4,01007,24330.0,23380.0,23160.0,22380.0,22146.451613,21986.500000,21679.333333
...,...,...,...,...,...,...,...,...
30926,97420,17480.0,16710.0,15840.0,15210.0,15110.666667,14622.500000,14125.555556
30927,97421,12960.0,12340.0,11880.0,11580.0,11280.937500,11019.291667,10827.419355
30928,97422,15880.0,15320.0,14880.0,14460.0,14243.809524,13798.871636,13349.444444
30929,97423,16250.0,15750.0,15090.0,14460.0,14031.785714,13693.500000,13104.516129


#### b) ZE level

In [11]:
df_rev_ze = filosofi_all_years("ZE")

In [12]:
df_rev_ze

,ZE,MED2020,MED2019,MED2018,MED2017,MED2016,MED2015,MED2014
0,0051,21030.0,20580.0,20360.0,19820.0,19596.000000,19317.833333,19125.500000
1,0052,20470.0,20000.0,19760.0,20060.0,19779.333333,19591.333333,19454.000000
2,0053,20820.0,20380.0,20070.0,21230.0,20985.857143,20765.238095,20573.913043
3,0054,22150.0,21750.0,21640.0,20070.0,19793.076923,19655.833333,19513.043478
4,0055,21320.0,20860.0,20800.0,19270.0,18956.666667,18794.666667,18657.333333
...,...,...,...,...,...,...,...,...
301,9403,20170.0,19760.0,19300.0,19550.0,18893.000000,18231.538462,17640.476190
302,9404,20540.0,20130.0,19770.0,19460.0,19062.666667,18755.000000,18376.400000
303,9405,20010.0,19340.0,18960.0,18900.0,18090.555556,17900.769231,17473.500000
304,9406,20610.0,20200.0,20110.0,19320.0,18689.333333,18686.000000,18426.000000


In [13]:
df_rev_ze.count()

ZE         306
MED2020    297
MED2019    297
MED2018    297
MED2017    111
MED2016    111
MED2015    111
MED2014    111
dtype: int64

There is an issue here because the income data for the years 2018, 2019, 2020 are given in the 2020 zone d'emploi division while the income data from 2014 to 2017 use the zone d'emploi division from 2010.

### Data extrapolation for the years 2021, 2022 and 2023

Income data for the years 2021, 2022 and 2023 are not available for now. Therefore, we need to extrapolate them based on income data from 2014 to 2020.

### Restructure dataframe

In [14]:
def melt_df(df, geo_level):
    df_melt = pd.melt(df, id_vars=[geo_level], var_name='Date', value_name='MED').sort_values(by=[geo_level, 'Date']).reset_index(drop=True)
    df_melt['Date'] = df_melt['Date'].str.extract(r'(\d+)')
    return df_melt

#### a) COM level

In [15]:
df_com = melt_df(df_rev_com, "COM")

In [16]:
df_com

,COM,Date,MED
0,01001,2014,21576.666667
1,01001,2015,22228.000000
2,01001,2016,22679.000000
3,01001,2017,23310.000000
4,01001,2018,25350.000000
...,...,...,...
216512,97424,2016,12034.375000
216513,97424,2017,12280.000000
216514,97424,2018,12420.000000
216515,97424,2019,12980.000000


#### b) ZE level

In [17]:
df_ze = melt_df(df_rev_ze, "ZE")

In [18]:
df_ze

,ZE,Date,MED
0,0051,2014,19125.500000
1,0051,2015,19317.833333
2,0051,2016,19596.000000
3,0051,2017,19820.000000
4,0051,2018,20360.000000
...,...,...,...
2137,9407,2016,18000.000000
2138,9407,2017,18450.000000
2139,9407,2018,20260.000000
2140,9407,2019,20000.000000


## 2- Population density

Population data by commune from 1876 to 2020 : https://www.insee.fr/fr/statistiques/fichier/3698339/base-pop-historiques-1876-2020.xlsx

Population data by zone d'emploi from 1876 to 2020 : https://www.observatoire-des-territoires.gouv.fr/historique-de-la-population-depuis-1876

Surface area by commune and zone d'emploi : https://www.observatoire-des-territoires.gouv.fr/superficie

#### a) COM level

In [19]:
def create_density_df_com():
    # Import population data
    pop = pd.read_excel(path+"/data/external/Density/base-pop-historiques-1876-2020.xlsx", sheet_name="pop_1876_2020", skiprows=5)
    pop = pop[['CODGEO', 'PMUN20', 'PMUN19', 'PMUN18', 'PMUN17', 'PMUN16', 'PMUN15', 'PMUN14']]
    
    # Import surface area
    superf = pd.read_excel(path+"/data/external/Density/insee_rp_hist_1968_superf_com.xlsx", sheet_name="Data", skiprows=4)
    superf = superf[['codgeo', 'superf_choro']].dropna().drop_duplicates().reset_index(drop=True)

    # Compute density = pop / surface area
    df = pop.merge(superf, left_on='CODGEO', right_on='codgeo', how='inner')
    for year in range(14, 21):
        pop_year = f'PMUN{year}'
        df[f'density_20{year}'] = df[pop_year] / df['superf_choro']

    df = df[['CODGEO', 'density_2020', 'density_2019', 'density_2018', 'density_2017', 'density_2016', 'density_2015', 'density_2014']]
    df = df.rename(columns={'CODGEO': 'COM'}) 
    
    # Restructure dataframe
    df = pd.melt(df, id_vars=['COM'], var_name='Date', value_name='density').sort_values(by=['COM', 'Date']).reset_index(drop=True)
    df['Date'] = df['Date'].str.extract(r'(\d+)')
    
    return df

In [20]:
df_dens_com = create_density_df_com()

In [21]:
df_dens_com

,COM,Date,density
0,01001,2014,48.087774
1,01001,2015,48.087774
2,01001,2016,48.087774
3,01001,2017,48.652038
4,01001,2018,48.338558
...,...,...,...
244477,97424,2016,64.016588
244478,97424,2017,64.644550
244479,97424,2018,65.071090
244480,97424,2019,65.616114


#### b) ZE level

In [22]:
def create_density_df_ze():
    # Import population data
    pop = pd.read_excel(path+"/data/external/Density/pop_hist_1876_ze.xlsx", sheet_name="Data", skiprows=4)
    pop = pop[(pop['an'] >= 2014) & (pop['an'] <= 2020)]
    pop['codgeo'] = pop['codgeo'].astype(str).str.zfill(4)
    
    # Import surface area
    superf = pd.read_excel(path+"/data/external/Density/insee_rp_hist_1968_superf_ze.xlsx", sheet_name="Data", skiprows=4)
    superf = superf[['codgeo', 'superf_choro']].dropna().drop_duplicates().reset_index(drop=True)
    superf['codgeo'] = superf['codgeo'].astype(str).str.zfill(4)

    # Compute density = pop / surface area
    df = pop.merge(superf, on='codgeo', how='inner')
    df['density'] = df['pop'] / df['superf_choro']
    df = df.rename(columns={'codgeo': 'ZE', 'an':'Date'}) 
    df['Date'] = df['Date'].astype(str) 
    df = df[['ZE', 'Date', 'density']]
    
    return df

In [23]:
df_dens_ze = create_density_df_ze()

In [24]:
df_dens_ze

,ZE,Date,density
0,0051,2014,54.249057
1,0051,2015,54.119318
2,0051,2016,53.877166
3,0051,2017,53.531900
4,0051,2018,53.165081
...,...,...,...
2193,9407,2016,15.907231
2194,9407,2017,15.910738
2195,9407,2018,15.819559
2196,9407,2019,15.866318
